###adult census dataset

work with raw dataset and then engineered dataset to observe the results

In [3]:
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_csv("adult.csv")
print(dataset.shape)
dataset.head()

(32561, 15)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [5]:
dataset.drop(["fnlwgt"], axis="columns", inplace=True)
dataset

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


### working with baseline data

In [6]:
base_df1 = dataset[~(dataset["workclass"].isin(["?"]) |
                     dataset["occupation"].isin(["?"])
                     )
]
base_df1.shape

(30718, 14)

splitting the data and encoding the variable

In [7]:
base_x = base_df1.drop(["income"], axis="columns")
base_y = base_df1["income"]

In [8]:
from sklearn.model_selection import train_test_split
bx_train, bx_test, by_train, by_test = train_test_split(base_x, base_y, test_size=0.2)

In [9]:
bx_train = pd.get_dummies(bx_train, drop_first=True)
bx_test = pd.get_dummies(bx_test, drop_first=True)
bx_train.replace({True: 1, False: 0}, inplace=True)
bx_test.replace({True: 1, False: 0}, inplace=True)
bx_train, bx_test = bx_train.align(bx_test, join='left', axis=1, fill_value=0)

/tmp/ipython-input-548629134.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bx_train.replace({True: 1, False: 0}, inplace=True)
/tmp/ipython-input-548629134.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bx_test.replace({True: 1, False: 0}, inplace=True)


In [10]:
by_train.replace({
    ">50K": 1,
    "<=50K": 0
}, inplace=True)
by_test.replace({
    ">50K": 1,
    "<=50K": 0
}, inplace=True)

/tmp/ipython-input-858639034.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  by_train.replace({
/tmp/ipython-input-858639034.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  by_test.replace({


### training and evaluating model

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

In [12]:
base_model = RandomForestClassifier(max_features='sqrt', min_samples_leaf=2, min_samples_split=10, n_estimators=100)
base_model.fit(bx_train, by_train)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=10)

In [13]:
base_pred = base_model.predict(bx_test)
base_prob = base_model.predict_proba(bx_test)[:,1]

In [14]:
base_acc = accuracy_score(by_test, base_pred)
base_prec = precision_score(by_test, base_pred)
base_recall = recall_score(by_test, base_pred)
base_roc = roc_auc_score(by_test, base_prob)

### feature engineering on dataset

In [15]:
dataset.head()

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [16]:
dataset.groupby(["occupation"])["income"].value_counts()

occupation         income
?                  <=50K     1652
                   >50K       191
Adm-clerical       <=50K     3263
                   >50K       507
Armed-Forces       <=50K        8
                   >50K         1
Craft-repair       <=50K     3170
                   >50K       929
Exec-managerial    <=50K     2098
                   >50K      1968
Farming-fishing    <=50K      879
                   >50K       115
Handlers-cleaners  <=50K     1284
                   >50K        86
Machine-op-inspct  <=50K     1752
                   >50K       250
Other-service      <=50K     3158
                   >50K       137
Priv-house-serv    <=50K      148
                   >50K         1
Prof-specialty     <=50K     2281
                   >50K      1859
Protective-serv    <=50K      438
                   >50K       211
Sales              <=50K     2667
                   >50K       983
Tech-support       <=50K      645
                   >50K       283
Transport-moving   <=50K     1277
                   >50K       320
Name: count, dtype: int64

In [17]:
dataset.groupby(["education"])["income"].value_counts()

education     income
10th          <=50K      871
              >50K        62
11th          <=50K     1115
              >50K        60
12th          <=50K      400
              >50K        33
1st-4th       <=50K      162
              >50K         6
5th-6th       <=50K      317
              >50K        16
7th-8th       <=50K      606
              >50K        40
9th           <=50K      487
              >50K        27
Assoc-acdm    <=50K      802
              >50K       265
Assoc-voc     <=50K     1021
              >50K       361
Bachelors     <=50K     3134
              >50K      2221
Doctorate     >50K       306
              <=50K      107
HS-grad       <=50K     8826
              >50K      1675
Masters       >50K       959
              <=50K      764
Preschool     <=50K       51
Prof-school   >50K       423
              <=50K      153
Some-college  <=50K     5904
              >50K      1387
Name: count, dtype: int64

In [18]:
dataset.groupby(["workclass"])["income"].value_counts()

workclass         income
?                 <=50K      1645
                  >50K        191
Federal-gov       <=50K       589
                  >50K        371
Local-gov         <=50K      1476
                  >50K        617
Never-worked      <=50K         7
Private           <=50K     17733
                  >50K       4963
Self-emp-inc      >50K        622
                  <=50K       494
Self-emp-not-inc  <=50K      1817
                  >50K        724
State-gov         <=50K       945
                  >50K        353
Without-pay       <=50K        14
Name: count, dtype: int64

In [19]:
dataset.groupby(["relationship"])["income"].value_counts()

relationship    income
Husband         <=50K     7275
                >50K      5918
Not-in-family   <=50K     7449
                >50K       856
Other-relative  <=50K      944
                >50K        37
Own-child       <=50K     5001
                >50K        67
Unmarried       <=50K     3228
                >50K       218
Wife            <=50K      823
                >50K       745
Name: count, dtype: int64

final observation

In [20]:
# native.country -> us/ non-us
# drop race
# education -> 1, 2, 3
# relationship -> primary, non-earner, dependent
# drop ? and minority values

In [21]:
df1 = dataset.drop(["race"], axis="columns")

In [22]:
df1["native.country"] = df1["native.country"].apply(lambda x: "US" if x=="United-States" else "Non-US")

In [23]:
def edu_flag(x):
    if x in ["Bachelors", "Masters", "Doctorate"]:
        return 3 # high
    elif x in ["Some-college", "Assoc-voc", "Assoc-acdm"]:
        return 2 # medium
    else:
        return 1 # low

df1["education"] = df1["education"].apply(edu_flag)

In [24]:
def rel_group(x):
    if x in ["Husband", "Wife"]:
        return "primary-earner"
    elif x in ['Not-in-family', 'Unmarried']:
        return "non-earner"
    else:
        return "dependent"

df1["relationship"] = df1["relationship"].apply(rel_group)
df1["relationship"]

,relationship
0,non-earner
1,non-earner
2,non-earner
3,non-earner
4,dependent
...,...
32556,non-earner
32557,primary-earner
32558,primary-earner
32559,non-earner


In [25]:
df1 = df1[~(df1['workclass'].isin(['?', 'Without-pay', 'Never-worked']) |
        df1['occupation'].isin(['Armed-Forces', '?'])
    )
]

### splitting into training and testing

In [26]:
x = df1.drop(columns="income")
y = df1["income"]

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [28]:
x_train = pd.get_dummies(x_train, drop_first=True)
x_test = pd.get_dummies(x_test, drop_first=True)
x_train.replace({True: 1, False: 0}, inplace=True)
x_test.replace({True: 1, False: 0}, inplace=True)
x_train, x_test = x_train.align(x_test, join='left', axis=1, fill_value=0)

/tmp/ipython-input-2566241371.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train.replace({True: 1, False: 0}, inplace=True)
/tmp/ipython-input-2566241371.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_test.replace({True: 1, False: 0}, inplace=True)


In [29]:
y_train.replace({
    ">50K": 1,
    "<=50K": 0
}, inplace=True)
y_test.replace({
    ">50K": 1,
    "<=50K": 0
}, inplace=True)

/tmp/ipython-input-545780699.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train.replace({
/tmp/ipython-input-545780699.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test.replace({


### training the model on engineered data

In [30]:
eng_model = RandomForestClassifier(max_features='sqrt', min_samples_leaf=2, min_samples_split=10, n_estimators=100)
eng_model.fit(x_train, y_train)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=10)

In [31]:
eng_pred = eng_model.predict(x_test)
eng_prob = eng_model.predict_proba(x_test)[:,1]

In [32]:
eng_acc = accuracy_score(y_test, eng_pred)
eng_prec = precision_score(y_test, eng_pred)
eng_recall = recall_score(y_test, eng_pred)
eng_roc = roc_auc_score(y_test, eng_prob)

### comparing scores

In [33]:
scores = []
scores.append({
    "data": "baseline",
    "accuracy": base_acc,
    "precision": base_prec,
    "recall": base_recall,
    "roc_auc": base_roc
})
scores.append({
    "data": "engineered",
    "accuracy": eng_acc,
    "precision": eng_prec,
    "recall": eng_recall,
    "roc_auc": eng_roc
})
result = pd.DataFrame(scores, columns=["data", "accuracy", "precision", "recall", "roc_auc"])
result

,data,accuracy,precision,recall,roc_auc
0,baseline,0.856608,0.772222,0.620932,0.914713
1,engineered,0.863170,0.751784,0.642712,0.915921


After feature engineering, the model's performance has seen improvement. Even if the scores with the baseline data are not that bad, it just shows that the engineered data model performed better even with lesser columns and data, which saves storage.

An improvement in accuracy and recall shows that model is predicting more positive values, rather than a fluctuation. There is a tradeoff between precision and recall which results in a better roc-auc score.

Feature engineering shows that a model can performs better, if not equal with baseline model, with lesser data, and data which contributes to the final reuslt.